In [ ]:
########################################
### imports   ##########################
########################################
from matplotlib.dates import DateFormatter, DayLocator, YearLocator
import matplotlib.lines as ml
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
from tsfresh.feature_extraction import extract_features,ComprehensiveFCParameters

In [ ]:
########################################
### list input files ###################
########################################
path = "../data"
data_files = glob.glob(path + "/*.xls*")

In [ ]:
########################################
### read data from source ##############
########################################
df_list = (pd.read_excel(file, parse_dates=True,sheet_name=1,na_values=["*","-"]) for file in data_files)
dfSource=  pd.concat(df_list, ignore_index=True)

In [ ]:
df=dfSource

In [ ]:
########################################
### add/rename columns    ##############
########################################
df=df.filter(items=["annee", "Code métier BMO","met"])
#df["annee"] = pd.to_datetime(df["annee"],  format='%Y')
df=df.rename(columns = {'met':'rawpop'})
df=df.rename(columns = {'Code métier BMO':'job_code'})
df=df.rename(columns = {'annee':'date'})
df=df.assign(logpop=lambda x: np.log(x.rawpop))


In [ ]:
########################################
### group data            ##############
########################################
groupedDf=df.groupby(by=["date","job_code"]).sum().sort_index()

In [ ]:
availables_years=groupedDf.index.get_level_values(0).drop_duplicates().tolist()

In [ ]:
########################################
### previous rawpop & growth   #########
########################################

def getPrevious(date,job_code):
    previous_date=date-1
    if(previous_date in availables_years and (previous_date,job_code) in  groupedDf.index):
        return groupedDf.loc[(previous_date,job_code),'rawpop']
    elif(previous_date not in  availables_years):
        return groupedDf.loc[(date,job_code),'rawpop']
    else:
        return 0
    
for i in groupedDf.index:
    index=groupedDf.loc[i]
    groupedDf.loc[i, 'prev_rawpop'] = getPrevious(index.name[0],index.name[1])

groupedDf=groupedDf.assign(growth=lambda x: x['rawpop']-x['prev_rawpop'])

In [ ]:
########################################
### is_emerging_job            #########
########################################
def is_emerging_job(date,job_code): 
    prev_rawpop=groupedDf.loc[(date,job_code),'prev_rawpop']
    growth=groupedDf.loc[(date,job_code),'growth']
    return prev_rawpop<=np.quantile(groupedDf.loc[date,'prev_rawpop'],.1) and  growth>= np.quantile(groupedDf.loc[date,'growth'],.1)
    
for i in groupedDf.index:
    index=groupedDf.loc[i]
    groupedDf.loc[i, 'is_emerging_job'] = 1 if (is_emerging_job(index.name[0],index.name[1])) else 0

In [ ]:
########################################
### Time serie                 #########
########################################
fig, ax = plt.subplots(layout='constrained')
fig.get_layout_engine().set(hspace=0.15)
ax.set_title("nb de projet de recrutement par an")

lns=[]
flatDf=groupedDf.reset_index()
jobs=flatDf['job_code'].drop_duplicates()
for job in jobs:
  ddf=flatDf.query('job_code==@job')
  ln = ax.plot(ddf['date'], ddf['rawpop'], label=job)
  lns+=ln

labels=[l.get_label() for l in lns]
#ax.legend(lns, labels)
ax.xaxis.set_major_locator(YearLocator())
ax.xaxis.set_major_formatter(DateFormatter('%Y'))

In [ ]:
########################################
### Add id column  = row number#########
########################################
df_with_id=groupedDf.reset_index()
df_with_id=df_with_id.assign(id=lambda x: range(1, 1+len(x)))

In [ ]:
########################################
### Extract features           #########
########################################
settings = ComprehensiveFCParameters()
featuredDF=extract_features(df_with_id,default_fc_parameters=settings, column_id='id')

In [ ]:
########################################
### Export dataset             #########
########################################
featuredDF.to_csv("../export/featured_dataset.csv",sep=";")